In [2]:
import geopandas as gpd
from pathlib import Path
import json
import pandas as pd
from sqlalchemy import URL, create_engine, text as sql_text

In [3]:
credentials = Path('../../inputs/db_credentials_old.json')
with open(credentials, 'r') as f:
    credentials = json.load(f)

user     = credentials['user']
password = credentials['password']
host     = credentials['host']
port     = credentials['port']
database = credentials['database']

In [4]:
url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

Execute a query

In [5]:
thuc_id = 1002

In [8]:
with engine.connect() as connection:
    try:
        query = f""" 
        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ADD COLUMN soil_id_annagnps_valid INT;

        UPDATE thuc_{thuc_id}_annagnps_cell_data_section
        SET soil_id_annagnps_valid = CASE WHEN soil_id IN (SELECT Soil_ID FROM usa_valid_soil_data) THEN 1 ELSE 0 END;

        """
        connection.execute(sql_text(query))
        # Commit the transaction explicitly
        connection.commit()

    except Exception as e:
        print(e)
        # Rollback the transaction in case of an error
        connection.rollback()


Check a query

In [9]:
query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_data_section"

with engine.connect() as conn:

    try:

        df = pd.read_sql_query(sql=sql_text(query), con=conn)

    except Exception as e:

        raise e

In [12]:
df[df['soil_id_annagnps_valid']==1]

,cell_id,soil_id,mgmt_field_id,reach_id,reach_location_code,cell_area,time_of_conc,avg_elevation,rcn_calib_id,avg_land_slope,...,delivery_ratio,constant_usle_c_fctr,constant_usle_p_fctr,all_oc_calib_fctr,all_n_calib_fctr,all_p_calib_fctr,sheet_and_rill_erosion_calib_fctr,gullies_erosion_calib_fctr,input_units_code,soil_id_annagnps_valid
0,83842,242963,Developed_Open_Space,8384,1,1.08,None,218.10,None,0.01139,...,None,None,None,None,None,None,None,None,1,1
1,83862,242966,Developed_Open_Space,8386,1,7.92,None,224.91,None,0.02158,...,None,None,None,None,None,None,None,None,1,1
2,83813,242965,Developed_Open_Space,8381,1,18.36,None,224.46,None,0.01026,...,None,None,None,None,None,None,None,None,1,1
3,83852,242963,Developed_Open_Space,8385,1,13.05,None,223.55,None,0.02035,...,None,None,None,None,None,None,None,None,1,1
4,83843,242963,Developed_Open_Space,8384,1,0.18,None,218.50,None,0.01317,...,None,None,None,None,None,None,None,None,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52045,207103,176757,Developed,20710,1,21.06,None,123.67,None,0.00458,...,None,None,None,None,None,None,None,None,1,1
52046,42,176757,Developed,4,1,36.81,None,123.49,None,0.00841,...,None,None,None,None,None,None,None,None,1,1
52047,23,1652030,Developed,2,1,0.72,None,122.99,None,0.01727,...,None,None,None,None,None,None,None,None,1,1
52048,22,176757,Developed,2,1,1.71,None,122.92,None,0.01867,...,None,None,None,None,None,None,None,None,1,1
